# Setup

In [1]:
import os
import glob
import cv2

from pylibdmtx.pylibdmtx import decode
from ultralytics import YOLO, settings
root_dir = os.getcwd().replace('\\notebooks', '')
settings.update({'datasets_dir': f'{root_dir}/data/processed/test', 'runs_dir': f'{root_dir}/yolo/runs'})

# Evaluation Functions

In [2]:
def eval_map(model, eval_yaml):
    '''
    Runs ultralytics val function to get mAP score and other metrics
    '''
    metrics = model.val(data=eval_yaml, split='test')

    print()
    print(f'Precision : {round(metrics.box.p[0], 2)}')  # close to 1 is good
    print(f'Recall    : {round(metrics.box.r[0], 2)}')  # close to 1 is good
    print(f'F1        : {round(metrics.box.f1[0], 2)}') # close to 1 is good
    print(f'mAP50-95  : {round(metrics.box.map, 3)}')   # 0.3 can be good, higher is better

    return

In [3]:
def crop_decoding(model, glob_path):
    '''
    Performs cropping and decoding of the images
    '''
    img_paths = glob.glob(glob_path)

    # stat tracking
    num_decodings = 0
    num_valid_decodings = 0

    idx = 0
    for img_path in img_paths:
        img = cv2.imread(img_path)
        results = model(img)
        boxes = results[0].boxes.xyxy.tolist()

        actual_decoding = os.path.basename(img_path).split('_')[0]
        actual_decoding = actual_decoding.replace('-', '') # minor cleaning

        # fixing some actual decodings
        if actual_decoding == 'KV8INMEP':
            actual_decoding = '#D1FPE50HA9NS0047XG264##KV8INMEP'
        elif actual_decoding == 'KW8PXY3D':
            actual_decoding = '#D1FPE50HA9NS0047XG264##KW8PXY3D'

        if boxes != None:
            for box in boxes:
                # crop with some padding (to not have too small of a crop)
                pad = 10
                crop_obj = img[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
                crop_obj = img[max(0, int(box[1])-pad):max(0, int(box[3])+pad), max(0, int(box[0])-pad):max(0, int(box[2])+pad)]
                decodings = decode(crop_obj)

                if decodings != None and len(decodings) > 0:
                    num_decodings += 1
                    
                    print(actual_decoding)
                    for decoding in decodings:
                        decoded_string = decoding.data.decode('utf-8')
                        print(decoded_string)
                        if decoded_string == actual_decoding:
                            print('Valid decoding!')
                            num_valid_decodings += 1
                        else:
                            print('Invalid decoding!')

                # optional saving
                cv2.imwrite(f'../data/cropped/{actual_decoding}-{idx}.jpg', crop_obj)
                idx += 1

    # calculate stats
    dm_decode_rate = num_decodings/len(img_paths)
    valid_decode_rate = num_valid_decodings/len(img_paths)

    print()
    print(f'{num_valid_decodings}/{len(img_paths)}')
    print(f'Dm decode rate: {dm_decode_rate}')
    print(f'Valid decode rate: {valid_decode_rate}')

    return

In [4]:
def eval_yolo(model_path, glob_path, eval_yaml):
    '''
    Given a yolo model, makes the following evaluations:
     - mAP scores for bounding boxes
     - DM decode rate (% of decodings of test images)
     - Valid DM decode rate (% of decodings of test images that match with the serial number)
    Also prints the first "print_count" images with predictions.
    '''
    # Load the model
    model = YOLO(model_path)

    # mAP scores (saves to runs dir)
    eval_map(model, eval_yaml)

    # Crop and decode
    crop_decoding(model, glob_path)

    return

In [5]:
def eval_baseline(glob_path):
    '''
    Runs baseline decoder and counts the number of decodings
    '''
    img_paths = glob.glob(glob_path)

    # stat tracking
    num_decodings = 0
    num_valid_decodings = 0

    for img_path in img_paths:
        img = cv2.imread(img_path)
        
        decodings = decode(img)

        actual_decoding = os.path.basename(img_path).split('_')[0]
        actual_decoding = actual_decoding.replace('-', '')

        # fixing some actual decodings
        if actual_decoding == 'KV8INMEP':
            actual_decoding = '#D1FPE50HA9NS0047XG264##KV8INMEP'
        elif actual_decoding == 'KW8PXY3D':
            actual_decoding = '#D1FPE50HA9NS0047XG264##KW8PXY3D'

        if decodings is not None and len(decodings) > 0:
            num_decodings += 1

            print(actual_decoding)
            for decoding in decodings:
                decoded_string = decoding.data.decode('utf-8')
                print(decoded_string)

                if decoded_string == actual_decoding:
                    print('Valid decoding!')
                    num_valid_decodings += 1
                else:
                    print('Invalid decoding!')
    
    # calculate stats
    dm_decode_rate = num_decodings/len(img_paths)
    valid_decode_rate = num_valid_decodings/len(img_paths)

    print()
    print(f'{num_valid_decodings}/{len(img_paths)}')
    print(f'Dm decode rate: {dm_decode_rate}')
    print(f'Valid decode rate: {valid_decode_rate}')

    return

# Baseline

In [6]:
eval_baseline('../data/MAN/images/test/*.jpg')

250115190210016
250115190210016
Valid decoding!
4F1K99136940006
4F1K99136940006
Valid decoding!
4F1K99136940007
4F1K99136940007
Valid decoding!
#D1FPE50HA9NS0047XG264##KV8INMEP
#D1FPE50HA9NS0047XG264##KV8INMEP
Valid decoding!
#D1FPE50HA9NS0047XG264##KW8PXY3D
#D1FPE50HA9NS0047XG264##KW8PXY3D
Valid decoding!

5/50
Dm decode rate: 0.1
Valid decode rate: 0.1


## Kaggle from Scratch

In [7]:
eval_yolo('../yolo/models/kaggle_scratch.pt', '../data/MAN/images/test/*.jpg', eval_yaml='../data/MAN/data.yaml')

Ultralytics 8.3.6  Python-3.12.5 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients


val: Scanning C:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\data\MAN\labels\test.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:06<00:00,  1.61s/it]


                   all         50         50      0.256       0.24      0.106      0.069
Speed: 5.2ms preprocess, 7.2ms inference, 0.0ms loss, 5.0ms postprocess per image
Results saved to c:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\yolo\runs\detect\val

Precision : 0.26
Recall    : 0.24
F1        : 0.25
mAP50-95  : 0.069

0: 640x640 6 Data Matrixs, 10.4ms
Speed: 2.0ms preprocess, 10.4ms inference, 5.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 14.9ms
Speed: 90.3ms preprocess, 14.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Data Matrix, 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Data Matrix, 12.4ms
Speed: 2.0ms preprocess, 12.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
250115190210016
250115190210016
Valid decoding!

0: 640x640 (no detections), 8.0ms
Speed

## Kaggle finetuned

In [8]:
eval_yolo('../yolo/models/kaggle_finetuned.pt', '../data/MAN/images/test/*.jpg', eval_yaml='../data/MAN/data.yaml')

Ultralytics 8.3.6  Python-3.12.5 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients


val: Scanning C:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\data\MAN\labels\test.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:07<00:00,  1.95s/it]


                   all         50         50      0.894      0.841      0.914      0.747
Speed: 5.2ms preprocess, 4.2ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to c:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\yolo\runs\detect\val2

Precision : 0.89
Recall    : 0.84
F1        : 0.87
mAP50-95  : 0.747

0: 640x640 1 Data Matrix, 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Data Matrix, 50.6ms
Speed: 3.0ms preprocess, 50.6ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Data Matrix, 69.6ms
Speed: 3.0ms preprocess, 69.6ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Data Matrix, 7.0ms
Speed: 2.3ms preprocess, 7.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
250115190210016
250115190210016
Valid decoding!

0: 640x640 1 Data Matrix, 9.0ms
Speed: 2.0ms

## Ultralytics finetuned

In [9]:
eval_yolo('../yolo/models/ultralytics_finetuned.pt', '../data/MAN/images/test/*.jpg', eval_yaml='../data/MAN/data.yaml')

Ultralytics 8.3.6  Python-3.12.5 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients


val: Scanning C:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\data\MAN\labels\test.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:07<00:00,  1.99s/it]


                   all         50         50      0.956        0.9      0.947      0.758
Speed: 7.2ms preprocess, 4.3ms inference, 0.0ms loss, 5.0ms postprocess per image
Results saved to c:\Users\aidan\OneDrive\Desktop\itu\msc\courses\sem3\research_project\Research-Project-Data-Matrix-Code-\yolo\runs\detect\val3

Precision : 0.96
Recall    : 0.9
F1        : 0.93
mAP50-95  : 0.758

0: 640x640 1 Data Matrix, 14.0ms
Speed: 2.0ms preprocess, 14.0ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Data Matrix, 64.1ms
Speed: 3.0ms preprocess, 64.1ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Data Matrix, 195.2ms
Speed: 2.9ms preprocess, 195.2ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Data Matrix, 9.0ms
Speed: 4.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
250115190210016
250115190210016
Valid decoding!

0: 640x640 1 Data Matrix, 12.0ms
Speed: 2.0